In [4]:
!pip install matplotlib numpy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import matplotlib.pyplot as plt
import numpy as np

def plot_grid_path(grid, path, title="Grid Path"):
    """
    Plots the grid and the agent's path.
    """
    # Convert grid data to a numpy array for easier plotting
    grid_data = np.array(grid.map_data)

    # Create a colormap: green for start, red for end, gray for obstacles, blue gradient for cost
    cmap = plt.cm.Blues
    cmap.set_bad(color='gray') # obstacles
    cmap.set_under(color='green') # start
    cmap.set_over(color='red') # end


    # Mask obstacles (value 99) for plotting
    masked_grid = np.ma.masked_where(grid_data == 99, grid_data)

    plt.figure(figsize=(8, 8))
    plt.imshow(masked_grid, cmap=cmap, origin='upper', vmin=0, vmax=np.max(grid_data[grid_data != 99]))

    # Plot start and end points
    plt.plot(grid.start[1], grid.start[0], 'go', markersize=10, label='Start')
    plt.plot(grid.end[1], grid.end[0], 'ro', markersize=10, label='End')

    # Plot the path
    if path:
        path_x = [p[1] for p in path]
        path_y = [p[0] for p in path]
        plt.plot(path_x, path_y, 'k-', linewidth=2, label='Path')

    plt.title(title)
    plt.xticks([])
    plt.yticks([])
    plt.legend()
    plt.grid(False)
    plt.show()

In [7]:
# Modify the run_experiment function to include plotting
def run_experiment(maps):
    """
    Runs each algorithm on the provided maps and prints results, including plotting.
    """
    print("--- Running Experiments ---")
    print("-" * 50)

    for map_name, map_data in maps.items():
        if map_name == "dynamic_map":
            continue # Handle dynamic map separately

        print(f"Map: {map_name} ({map_data['description']})")

        # Test BFS
        grid_bfs = Grid(map_data['data'], map_data['start'], map_data['end'])
        path_bfs, cost_bfs, nodes_bfs, time_bfs = bfs(grid_bfs)
        if path_bfs:
            print(f"  > BFS: Cost={cost_bfs}, Nodes Expanded={nodes_bfs}, Time={time_bfs:.4f}s")
            plot_grid_path(grid_bfs, path_bfs, title=f"{map_name} - BFS Path")
        else:
            print(f"  > BFS: No path found.")

        # Test A*
        grid_a_star = Grid(map_data['data'], map_data['start'], map_data['end'])
        path_a_star, cost_a_star, nodes_a_star, time_a_star = a_star(grid_a_star)
        if path_a_star:
            print(f"  > A*: Cost={cost_a_star}, Nodes Expanded={nodes_a_star}, Time={time_a_star:.4f}s")
            plot_grid_path(grid_a_star, path_a_star, title=f"{map_name} - A* Path")
        else:
            print(f"  > A*: No path found.")

        print("-" * 50)

# Modify the run_dynamic_replanning_demo function to include plotting
def run_dynamic_replanning_demo(map_data):
    """
    Demonstrates the dynamic replanning functionality, including plotting.
    """
    print("--- Dynamic Replanning Demo ---")
    print(f"Map: dynamic_map ({map_data['description']})")

    # Initial plan with A*
    grid = Grid(map_data['data'], map_data['start'], map_data['end'], map_data['dynamic_obstacles'])
    initial_path, initial_cost, _, _ = a_star(grid)

    if not initial_path:
        print("Initial path not found. Cannot run demo.")
        return

    agent = Agent(grid)
    agent.path = initial_path[1:]  # Start at the second step

    print(f"Time {agent.time_step}: Agent at start {agent.current_pos}. Initial plan cost={initial_cost}.")
    print(f"Time {agent.time_step}: Initial Path: {initial_path}")
    plot_grid_path(grid, initial_path, title="Dynamic Map - Initial A* Path")


    while agent.current_pos != agent.grid.end:
        print(f"Time {agent.time_step}: Agent at {agent.current_pos}.")

        # Check for dynamic obstacle and replan
        replanned = dynamic_replanning(agent)
        if replanned:
            # Plot the new path after replanning
            plot_grid_path(agent.grid, [agent.current_pos] + agent.path, title=f"Dynamic Map - Replanned Path at Time {agent.time_step}")


        # Move the agent
        agent.move()
        time.sleep(0.1)  # Simulate a time step

        if not agent.path and agent.current_pos != agent.grid.end:
            print(f"Time {agent.time_step}: Agent is stuck. No path to goal.")
            break

    print(f"Time {agent.time_step}: Agent has arrived at destination {agent.current_pos}.")
    print("--- Demo Complete ---")

# Re-run the main execution block
if __name__ == "__main__":
    while True:
        maps = get_maps()

        # Allow user to select a map
        print("Available maps:")
        for i, map_name in enumerate(maps.keys()):
            print(f"{i+1}. {map_name} ({maps[map_name]['description']})")

        while True:
            try:
                print("\nEnter your map choice below:") # Added print statement
                map_choice = int(input(f"Enter the number of the map you want to use (1-{len(maps)}): ")) - 1
                selected_map_name = list(maps.keys())[map_choice]
                selected_map_data = maps[selected_map_name]
                break
            except (ValueError, IndexError):
                print("Invalid input. Please enter a valid map number.")

        # Allow user to define dynamic obstacles for the selected map
        if selected_map_name == "dynamic_map":
            print("\nDefining dynamic obstacles for dynamic_map:")
            dynamic_obstacles_input = {}
            while True:
                try:
                    print("\nEnter the number of dynamic obstacles:") # Added print statement
                    num_obstacles = int(input("Enter the number of dynamic obstacles to add (enter 0 if none): "))
                    break
                except ValueError:
                    print("Invalid input. Please enter a number.")

            for i in range(num_obstacles):
                while True:
                    try:
                        print(f"\nEnter details for obstacle {i+1}:") # Added print statement
                        time_step = int(input(f"Enter the time step for obstacle {i+1}: "))
                        row = int(input(f"Enter the row for obstacle {i+1}: "))
                        col = int(input(f"Enter the column for obstacle {i+1}: "))
                        if time_step not in dynamic_obstacles_input:
                            dynamic_obstacles_input[time_step] = []
                        dynamic_obstacles_input[time_step].append((row, col))
                        break
                    except ValueError:
                        print("Invalid input. Please enter numbers for time step, row, and column.")

        selected_map_data["dynamic_obstacles"] = dynamic_obstacles_input

        # Run the selected scenario
        if selected_map_name == "dynamic_map":
            run_dynamic_replanning_demo(selected_map_data)
        else:
            # For non-dynamic maps, run both BFS and A* experiments
            temp_maps = {selected_map_name: selected_map_data}
            run_experiment(temp_maps)

        print("\nDo you want to run another experiment?") # Added print statement
        run_again = input("Do you want to run another experiment? (yes/no): ").lower()
        if run_again != 'yes':
            break

NameError: name 'get_maps' is not defined